##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

### Install dependencies

In [1]:
%pip install -U -q "google-genai>=1.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.8.0 which is incompatible.


In [2]:
from google import genai
from google.genai import types

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [5]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

This example will be based on the transcript from the Apollo 11 mission.

Start by downloading that transcript.

In [6]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


Now upload the transcript using the [File API](../quickstarts/File_API.ipynb).

In [39]:
document = client.files.upload(file="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the [`system_instruction`](../quickstarts/System_instructions.ipynb) has been set, and the document was provided in the prompt.

In [60]:
# Note that caching requires a frozen model, e.g. one with a `-001` suffix.
model = "gemini-1.5-flash-002"
from google import genai
from google.genai import types
apollo_cache = client.caches.create(
      model=model,
      config={
          'contents': [document],
          'system_instruction': 'You are an expert at analyzing transcripts.',
      },
  )

apollo_cache

CachedContent(name='cachedContents/wuk2sw5gno4y', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 11, 15, 35, 28, 654879, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 11, 15, 35, 28, 654879, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 11, 16, 35, 27, 375464, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [61]:
from google.genai import types

client.caches.update(
    name=apollo_cache.name,
    config=types.UpdateCachedContentConfig(ttl="7200s")  # 2 hours in seconds
)
apollo_cache = client.caches.get(name=apollo_cache.name) # Get the updated cache
apollo_cache

CachedContent(name='cachedContents/wuk2sw5gno4y', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 11, 15, 35, 28, 654879, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 11, 15, 36, 4, 88668, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 11, 17, 36, 4, 82843, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [97]:
response = client.models.generate_content(
    model=model,
    contents='Find a lighthearted moment from this transcript',
    config=types.GenerateContentConfig(
        cached_content=apollo_cache.name,
    )
)

print(response.text)

There are several lighthearted moments, depending on your definition of "lighthearted." Here are a couple of examples with slightly different tones:

**Example 1 (Playful Banter):**

At 00:00:05:35, CDR Armstrong makes a comment about the clarity of the communication from Bruce (the Capsule Communicator):  "You sure sound clear down there, Bruce. Sounds like you're sitting in your living room."  Bruce's reply, "Oh, thank you. You all are coming through beautifully, too," is a polite but equally playful response. This is a lighthearted exchange of pleasantries, contrasting the seriousness of the mission.

**Example 2 (Humor amidst Tension):**

At 01:01:29:27, LMP Aldrin uses a playful, self-deprecating nickname:  "Cecil B. deAldrin is standing by for instructions." This shows a touch of humor in the midst of technical challenges with the TV transmission.


The transcript contains other moments of camaraderie and informal language, but these two stand out as clearly intended to be lighth

You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [98]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=249, prompt_token_count=323392, total_token_count=323641)

You can ask new questions of the model, and the cache is reused.

In [99]:
chat = client.chats.create(model=model, config={"cached_content": apollo_cache.name})

response = chat.send_message(message="Give me a quote from the most important part of the transcript.")
print(response.text)

The most important part of the transcript is the moment of the Apollo 11 lunar module's landing.  The quote from that moment is:

"Houston, Tranquility Base here. The Eagle has landed."



In [100]:
response = chat.send_message(message="What was recounted after that?", config={"cached_content": apollo_cache.name})
print(response.text)

Immediately following the "The Eagle has landed" transmission, the following exchanges occurred:

* **Ground Control:** "Roger, Tranquility. We copy you on the ground. You got a bunch of guys about to turn blue. We're breathing again. Thanks a lot."  This expresses immense relief and joy at Mission Control.

* **Armstrong:** "Thank you." A brief, understated acknowledgement of the momentous achievement.

* **Ground Control:** "You're looking good here."  A further expression of relief and confirmation of the successful landing.

* **Aldrin:**  "Tranquility Base here ... MASTER ARM, ON. Take care of the ... I'll get this ..."  Aldrin begins reporting on post-landing procedures and systems checks.  The transmission is cut off mid-sentence.

* **Aldrin:** "... Very smooth touchdown."  A later comment describing the quality of the landing.

* **Aldrin:** "... It looks like we're venting the oxidizer now."  Aldrin reports on the venting of the oxidizer from the lunar module's descent stage.

In [101]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=334, prompt_token_count=323449, total_token_count=323783)

As you can see, among the 323455 tokens, 323383 were cached (and thus less expensive) and only 236 were from the prompt.

Since the cached tokens are cheaper than the normal ones, it means this prompt was 75% cheaper that if you had not used caching. Check the [pricing here](https://ai.google.dev/pricing) for the up-to-date discount on cached tokens.

## Counting tokens

The `GenerativeModel` object can be used to count the tokens of a request in the same manner as a direct, uncached, model.

In [109]:
response = client.models.count_tokens(
    model=model,
    contents="How many people are involved in this transcript?",
)

response


CountTokensResponse(total_tokens=10, cached_content_token_count=None)

## Delete the cache

The cache has a small recurring storage cost (cf. [pricing](https://ai.google.dev/pricing)) so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 2h.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [111]:
print(apollo_cache.name)
client.caches.delete(name=apollo_cache.name)

cachedContents/wuk2sw5gno4y


DeleteCachedContentResponse()

## Next Steps
### Useful API references:

If you want to know more about the caching API, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

### Continue your discovery of the Gemini API

Check the File API notebook to know more about that API. The [vision capabilities](../quickstarts/Video.ipynb) of the Gemini API are a good reason to use the File API and the caching.
The Gemini API also has configurable [safety settings](../quickstarts/Safety.ipynb) that you might have to customize when dealing with big files.
